In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.regularizers import L2
from tensorflow.keras.activations import relu,linear
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score,f1_score
from sklearn.preprocessing import StandardScaler

In [2]:
df=pd.read_csv("heart.csv")
X=df.iloc[:,:11]
y=df.iloc[:,11]
one_hot_variables=["Sex","ChestPainType","RestingECG","ExerciseAngina","ST_Slope"]
X=pd.get_dummies(data=X,prefix=one_hot_variables,columns=one_hot_variables)
X.shape

(918, 20)

In [29]:
X_norm=StandardScaler().fit_transform(X)
X_train,X_test,y_train,y_test=train_test_split(X_norm,y,train_size=0.8,random_state=42)


In [25]:
np.random.seed(42)
tf.random.set_seed(42)
model = Sequential([
    tf.keras.Input(shape=(20,)),
    Dense(units=20, name="L1", activation="relu",kernel_regularizer=L2(0.0099),),
    Dense(units=12, name="L2", activation="relu",kernel_regularizer=L2(0.0099)),
    Dense(units=12, name="L3", activation="relu",kernel_regularizer=L2(0.0099)),
    Dense(units=20, name="L4", activation="relu",kernel_regularizer=L2(0.0099)),
    Dense(units=1, name="L5", activation="linear",kernel_regularizer=L2(0.0099)),
    
],name="model_1")


In [5]:
model.summary()

Model: "model_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ L1 (Dense)                      │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L2 (Dense)                      │ (None, 12)             │           252 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L3 (Dense)                      │ (None, 12)             │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L4 (Dense)                      │ (None, 20)             │           260 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L5 (Dense)                      │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,109 (4.33 KB)

 Trainable params: 1,109 (4.33 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
model.compile(loss=BinaryCrossentropy(from_logits=True),metrics=[tf.keras.metrics.BinaryAccuracy()],optimizer=Adam(learning_rate=0.005))

In [31]:
model.fit(X_train,y_train,validation_data=[X_test,y_test],epochs=250, batch_size=32,verbose=1,shuffle=False)

Epoch 1/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - binary_accuracy: 0.8982 - loss: 0.4077 - val_binary_accuracy: 0.8750 - val_loss: 0.4509
Epoch 2/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.9064 - loss: 0.3622 - val_binary_accuracy: 0.8370 - val_loss: 0.4702
Epoch 3/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.9010 - loss: 0.3686 - val_binary_accuracy: 0.8370 - val_loss: 0.4686
Epoch 4/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8975 - loss: 0.3662 - val_binary_accuracy: 0.8478 - val_loss: 0.4635
Epoch 5/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8954 - loss: 0.3641 - val_binary_accuracy: 0.8533 - val_loss: 0.4603
Epoch 6/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.8950 - loss: 0.3622 - val_binary_accuracy: 0.8533 - val_loss: 0.4600
Epoch 7/250
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - binary_accuracy: 0.9000 - loss: 0.3599 - val_binary_accuracy: 0.8533 - val_loss: 0.4605
Epoch 8/250


In [21]:


loss, accuracy = model.evaluate(X_test, y_test)
print("Befor converting from linear to sigmoid:")
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8769 - loss: 0.4486 
Befor converting from linear to sigmoid:
Test Loss: 0.44961580634117126
Test Accuracy: 0.885869562625885


In [32]:

yhat_train=model.predict(X_train)
yhat_test=model.predict(X_test)
yhat_train=tf.nn.sigmoid(yhat_train)
yhat_test=tf.nn.sigmoid(yhat_test)
yhat_train_binary=(yhat_train>=0.5).numpy().astype(int)
yhat_test_binary=(yhat_test>=0.5).numpy().astype(int)
yhat_train_accuracy=accuracy_score(y_train,yhat_train_binary)
yhat_test_accuracy=accuracy_score(y_test,yhat_test_binary)
yhat_train_f1score=f1_score(y_train,yhat_train_binary)
yhat_test_f1score=f1_score(y_test,yhat_test_binary)
print("After converting from linear to sigmoid:")
print(f"Accuracy of trained data={yhat_train_accuracy*100:0.2f}%")
print(f"Accuracy of tested data={yhat_test_accuracy*100:0.2f}%")
print(f"F1 score of trained data={yhat_train_f1score*100:0.2f}%")
print(f"F1 score of tested data={yhat_test_f1score*100:0.2f}%")


23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
After converting from linear to sigmoid:
Accuracy of trained data=93.46%
Accuracy of tested data=91.30%
F1 score of trained data=94.00%
F1 score of tested data=92.23%
